In [23]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
import zipfile

In [24]:
sys.path.append('F:\study\ml\python_packages')
import d2l

In [3]:
def load_data_jay_lyrics(): 
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zif:
        with zif.open('jaychou_lyrics.txt') as f:
            corpus_chars =f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    
    return corpus_indices,char_to_idx,idx_to_char,vocab_size

In [4]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

In [5]:
def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens,dtype=torch.float32),
                                  requires_grad=True))
    
    W_xz,W_hz,b_z=_three()
    W_xr,W_hr,b_r=_three()
    W_xh,W_hh,b_h=_three()
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,dtype=torch.float32),
                          requires_grad=True)
    return nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q])
    

In [6]:
def init_gru_state(batch_size,num_hiddens):
    return (torch.zeros((batch_size,num_hiddens)),)

In [7]:
def gru(inputs,state,params):
    W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for X in inputs:
        ## 计算更新门
        Z=torch.sigmoid(torch.matmul(X,W_xz) + torch.matmul(H,W_hz)+b_z)
        ## 计算重置门
        R=torch.sigmoid(torch.matmul(X,W_xr) + torch.matmul(H,W_hr)+b_r)
        ## 计算候选隐藏状态
        H_tilda=torch.tanh(torch.matmul(X,W_xh)+R*torch.matmul(H,W_hh)+b_h)
        H=Z*H+(1-Z) * H_tilda
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)
        

In [8]:
num_epochs,num_steps,batch_size,lr,clipping_theta=240,50,32,1e2,1e-2
pred_period,pred_len,prefixes=40,50,['分开','不分开']

In [9]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

In [10]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    ## 除了第一个字,剩下都要输出
    for t in range(num_chars+len(prefix)-1):
        X=torch.tensor([output[-1]]).view(1,1)
        if state is not None:
            if isinstance(state,tuple):
                print('state shape2 : ',state.shape)
                print('state[0] : ',state[0])
                print('state[1] : ',state[1])
                state=(state[0],state[1])
            else:
                state=state
        (Y,state)=model(X,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [15]:
def train_and_predict_rnn(rnn, get_params, init_run_state, num_hiddens,
                          vocab_size, corpus_indices, idx_to_char, 
                          char_to_idx,
                          is_random_iter, num_epochs,num_steps, lr, clipping_theta,
                          batch_size, pred_period, pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive

    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_run_state(batch_size, num_hiddens)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_run_state(batch_size, num_hiddens)
            else:
                for s in state:
                    s.detach_()
#             print('----')
#             print('X : ',X)
#             print('X shape : ',X.shape)
            inputs = to_onehot(X, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            ##print()
            outputs = torch.cat(outputs, dim=0)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(outputs, y.long())

            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta)
            d2l.sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d,perplexity %f,time %.2f sec' %
                  (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(
                    ' -',
                    predict_run(prefix, pred_len, gru, params, init_run_state,
                                num_hiddens, vocab_size, idx_to_char,
                                char_to_idx))

In [16]:
def predict_run(prefix,num_chars,rnn,params,init_run_state,num_hiddens,vocab_size,idx_to_char,char_to_idx):
    state=init_run_state(1,num_hiddens)
    output=[char_to_idx[prefix[0]]] ## 分
    for t in range(num_chars + len(prefix)-1):
#         print('t : ',t)
#         print(output)
        X=to_onehot(torch.tensor([[output[-1]]]),vocab_size)
#         print('X: ',X)
        # 计算输出和更新隐藏状态
        (Y,state)=rnn(X,state,params)
#         print('output: ',output)
        # 下一个时间的输入是prefix里字符创或当前的追加预测字符
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
#             print('Y:\n',Y)
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])
        

In [17]:
def one_hot(x, n_class, dtype=torch.float32):
    x=x.long()
    return torch.zeros(x.shape[0], n_class,
                       dtype=dtype).scatter_(1, x.view(-1, 1), 1)

In [18]:
def to_onehot(X,n_class):
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])] 

In [19]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps):
    corpus_indices = torch.Tensor(corpus_indices)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0:batch_size * batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i:i + num_steps]
        Y = indices[:, i + 1:i + num_steps + 1]
        yield X, Y

In [20]:
def grad_clipping(params,theta):
    norm=torch.tensor([0.0])
    for param in params:
        norm+=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta/norm)
    

In [21]:
train_and_predict_rnn(gru,get_params,init_gru_state,num_hiddens,
                             vocab_size,corpus_indices,idx_to_char,
                              char_to_idx,False,num_epochs,num_steps,
                              lr,clipping_theta,batch_size,pred_period,
                              pred_len,prefixes
                             )

epoch 40,perplexity 206.438254,time 1.96 sec
 - 分开 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 不分开 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
epoch 80,perplexity 70.720872,time 1.96 sec
 - 分开 我不要再想 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
 - 不分开 我不要再想 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
epoch 120,perplexity 17.427691,time 1.92 sec
 - 分开 我想要你的微笑 一定元元演 你在人美前 你爱在美主 我爱你 我爱你的微笑 一定元人演 你在人美前 
 - 不分开 我爱你的让你在西元元 深埋在美索不达米亚平原 我 你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人
epoch 160,perplexity 4.088761,time 2.12 sec
 - 分开我心无处可爱躲 我坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
 - 不分开你 没有一直酒 慢慢在停留 你在它美留的 为什么我女朋友场外加 我想揍你已经很久 别想我 说你眼睛看
epoch 200,perplexity 1.547964,time 2.07 sec
 - 分开 我想要你的微笑每天都能看到  我知道这里很美但家乡的你更美走过我只想要你 陪我去吃汉堡  说穿了其
 - 不分开汉 我爱过这样打我不住 我不能再想你 我不要再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承
epoch 240,perplexity 1.093362,time 1.93 sec
 - 分开 我弄轻的生活 你说悔不乡泪友命纯 用铅笔用铅悉 纪录那最原始的美丽 纪录第一次遇见的你 如果我遇见
 - 不分开觉 我爱过这样友 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 


In [22]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super().__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        print('dense  : ',self.dense)
        self.state=None
    
    def forward(self,inputs,state): #input:(batch,seq_len)
        X=d2l.to_onehot(inputs,self.vocab_size)
        print('X[0] shape : ',X[0].shape)
        
        ## torch.stack 把 list(tensor1,tensor2)变成一个大的tensor
        ## 返回的Y 的特征为是 num_hidden
        
        Y,self.state=self.rnn(torch.stack(X),state)
        print('Y shape : ',Y.shape)
        print('state shape : ',self.state.shape)
        
        ## output 的特征纬度 是vocab_size
        output=self.dense(Y.view(-1,Y.shape[-1]))
        print('output shape : ',output.shape)
        return output,self.state

In [25]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size,
                                  corpus_indices, idx_to_char, char_to_idx,
                                  num_epochs, num_steps, lr, clipping_theta,
                                  batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size,
                                          num_steps)
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state, tuple):
                    print('----state is tuple----')
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()
            (output, state) = model(X, state)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())

            optimizer.zero_grad()
            l.backward()
            d2l.grad_clipping(model.parameters(), clipping_theta)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

            try:
                perplexity = math.exp(l_sum / n)
            except OverflowError:
                perplexity = float('inf')
            if (epoch + 1) % pred_period == 0:
                print('epoch %d ,perplexity %f,time %.2f sec' %
                      (epoch + 1, perplexity, time.time() - start))
                for prefix in prefixes:
                    print(
                        ' -',
                        predict_rnn_pytorch(prefix, pred_len, model,
                                            vocab_size, idx_to_char,
                                            char_to_idx))

In [20]:
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, corpus_indices,
                              idx_to_char, char_to_idx, num_epochs, num_steps,
                              lr, clipping_theta, batch_size, pred_period,
                              pred_len, prefixes)

dense  :  Linear(in_features=256, out_features=1027, bias=True)
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] s

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 25

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 25

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 25

X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
X[0] shape :  torch.Size([32, 1027])
Y shape :  torch.Size([50, 32, 25

...

Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
epoch 240 ,perplexity 1.079433,time 0.83 sec
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size(

Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
epoch 240 ,perplexity 1.077396,time 1.35 sec
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size(

Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])


Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
epoch 240 ,perplexity 1.074187,time 2.62 sec
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size(

Y shape :  torch.Size([50, 32, 256])
state shape :  torch.Size([1, 32, 256])
output shape :  torch.Size([1600, 1027])
epoch 240 ,perplexity 1.073684,time 3.12 sec
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size([1, 1, 256])
output shape :  torch.Size([1, 1027])
X[0] shape :  torch.Size([1, 1027])
Y shape :  torch.Size([1, 1, 256])
state shape :  torch.Size(